In [1]:
import numpy as np
import statsmodels.api as sm

# Datos de entrada
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])

# Agregar constante (para incluir la ordenada al origen)
X = sm.add_constant(x)  # Esto agrega una columna de unos para el término constante

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Mostrar los resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     16.35
Date:                Sun, 08 Dec 2024   Prob (F-statistic):             0.0272
Time:                        17:24:46   Log-Likelihood:                -21.674
No. Observations:                   5   AIC:                             47.35
Df Residuals:                       3   BIC:                             46.57
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.441e-15     10.661  -4.17e-16      1.0

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [21]:
import numpy as np

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])

# Varianza del error
sigma_squared = 100

# Calcular x^T x
xtx = np.sum(x**2)

# Calcular beta sombrero
beta_hat = np.sum(x * y) / xtx

# Calcular la varianza de beta sombrero
var_beta_hat = sigma_squared / xtx

# Resultados
{
    "beta_hat": beta_hat,
    "var_beta_hat": var_beta_hat
}


{'beta_hat': 6.5, 'var_beta_hat': 0.45454545454545453}

In [24]:
y_hat = beta_hat * x 
e_hat = y - y_hat
n = len(x)
k = 1
sigma_hat = np.sum(e_hat**2) / (n-k)
sigma_hat

426.25

In [26]:
y_hat

array([ 19.5,  45.5, -26. ,  32.5, -71.5])

In [25]:
sigma_hat / xtx

1.9375

In [20]:
np.sum(x * y)

1430

In [16]:
import numpy as np

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Varianza del error
sigma_squared = 100

# Paso 1: Regresión de x sobre z
ztz = np.sum(z**2)  # z^T z
ztx = np.sum(z * x)  # z^T x
pi_hat = ztx / ztz   # Coeficiente estimado

# Calcular x sombrero
x_hat = pi_hat * z

# Paso 2: Regresión de y sobre x sombrero
x_hat_t_x_hat = np.sum(x_hat**2)  # (x sombrero)^T x sombrero
x_hat_t_y = np.sum(x_hat * y)     # (x sombrero)^T y
beta_hat_2sls = x_hat_t_y / x_hat_t_x_hat

# Calcular la varianza de beta sombrero
var_beta_hat_2sls = sigma_squared / x_hat_t_x_hat

# Resultados
{
    "pi_hat (coeficiente x sobre z)": pi_hat,
    "x_hat": x_hat,
    "beta_hat_2sls": beta_hat_2sls,
    "var_beta_hat_2sls": var_beta_hat_2sls
}


{'pi_hat (coeficiente x sobre z)': 2.0,
 'x_hat': array([  2.,   4.,  -4.,   8., -10.]),
 'beta_hat_2sls': 7.2,
 'var_beta_hat_2sls': 0.5}

In [17]:
var_beta_hat = 0.45454545454545453
var_beta_hat_2sls = 0.5

(6.5 - 7.2) / np.sqrt(var_beta_hat_2sls - var_beta_hat)

-3.283291031876401

In [37]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import chi2

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Paso 1: Estimación por OLS sin constante
model_ols = sm.OLS(y, x).fit()  # OLS sin constante
beta_ols = model_ols.params
var_beta_ols = 0.45454545

# Paso 2: Estimación por 2SLS sin constante
# Etapa 1: Regresión de x sobre z (sin constante)
model_stage1 = sm.OLS(x, z).fit()
x_hat = model_stage1.predict(z)

# Etapa 2: Regresión de y sobre x sombrero (sin constante)
model_2sls = sm.OLS(y, x_hat).fit()
beta_2sls = model_2sls.params
var_beta_2sls = 0.5

# Paso 3: Estadístico de Hausman
beta_diff = beta_ols - beta_2sls  # Diferencia entre los estimadores
var_diff =  var_beta_2sls - var_beta_ols  # Diferencia entre varianzas
hausman_stat = beta_diff.T * var_diff * beta_diff  # Estadístico

# Usar scipy.stats.chi2.sf para calcular el p-valor
p_value = 1 - chi2.sf(hausman_stat, df=1)  # p-valor, 1 parámetro

# Resultados
{
    "beta_ols": beta_ols,
    "beta_2sls": beta_2sls,
    "hausman_stat": hausman_stat,
    "p_value": p_value
}



{'beta_ols': array([6.5]),
 'beta_2sls': array([7.2]),
 'hausman_stat': array([0.02227273]),
 'p_value': array([0.11863614])}

In [47]:
from scipy.stats import chi2

# Grados de libertad
df = 2  # Cambia según el número de parámetros que estás probando

# Punto crítico al 5% en la cola derecha
chi2_critical = chi2.ppf(0.95, df)

print(f"Punto crítico al 5% (chi-cuadrado, df={df}): {chi2_critical}")


Punto crítico al 5% (chi-cuadrado, df=2): 5.991464547107979


In [8]:
from scipy.stats import norm, t

# Grados de libertad para t-Student
df = 1  # Cambia según el número de grados de libertad

# Punto crítico al 5% en la cola derecha para normal
z_critical = norm.ppf(0.95)  # Normal estándar (media=0, desviación estándar=1)

# Punto crítico al 5% en la cola derecha para t-Student
t_critical = t.ppf(0.95, 1000)

print(f"Punto crítico al 5% (normal estándar): {z_critical}")
print(f"Punto crítico al 5% (t-Student, df={df}): {t_critical}")


Punto crítico al 5% (normal estándar): 1.6448536269514722
Punto crítico al 5% (t-Student, df=1): 1.6463788172854639


In [14]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Estimación por OLS sin constante (sin agregar constante)
model_ols = sm.OLS(y, x).fit()  # OLS sin constante
beta_ols = model_ols.params[0]  # Coeficiente estimado por OLS
var_beta_ols = model_ols.cov_params()[0, 0]  # Varianza del estimador OLS

# Estimación por 2SLS sin constante
# Etapa 1: Regresión de x sobre z (sin constante)
model_stage1 = sm.OLS(x, z).fit()
x_hat = model_stage1.predict(z)

# Etapa 2: Regresión de y sobre x sombrero (sin constante)
model_2sls = sm.OLS(y, x_hat).fit()
beta_2sls = model_2sls.params[0]  # Coeficiente estimado por 2SLS
var_beta_2sls = model_2sls.cov_params()[0, 0]  # Varianza del estimador 2SLS

# Estimación por MCC
# Para ilustrar, tomamos los mismos resultados de OLS como MCC (esto es solo un ejemplo)
beta_mcc = beta_ols
var_beta_mcc = var_beta_ols

# Calcular el estadístico de Hausman
beta_diff = beta_ols - beta_2sls  # Diferencia entre los estimadores
var_diff = var_beta_2sls - var_beta_mcc  # Diferencia entre las varianzas (2SLS - MCC)

# Estadístico de Hausman
hausman_stat = beta_diff / np.sqrt(var_diff)

# Calcular el p-valor usando la distribución normal estándar (asumiendo que sigue N(0,1))
p_value = 1 - norm.cdf(hausman_stat)  # p-valor

# Resultados
{
    "beta_ols": beta_ols,
    "beta_2sls": beta_2sls,
    "hausman_stat": hausman_stat,
    "p_value": p_value
}

C:\Users\HP\AppData\Local\Temp\ipykernel_13664\2666195302.py:35: RuntimeWarning: invalid value encountered in sqrt
  hausman_stat = beta_diff / np.sqrt(var_diff)


{'beta_ols': 6.5, 'beta_2sls': 7.2, 'hausman_stat': nan, 'p_value': nan}

In [4]:
import numpy as np
import statsmodels.api as sm

# Datos de entrada
x = np.array([3, 7, -4, 5, -11])
z = np.array([2, 4, -4, 8, -10])
y = np.array([20, 30, -50, 60, -60])

# Crear matriz de diseño con x y z como variables explicativas
X = np.column_stack((x, z))  # Combina x y z en una matriz
# X = sm.add_constant(X)  # Agregar constante al modelo

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Mostrar los resultados
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.943
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              24.82
Date:                Mon, 09 Dec 2024   Prob (F-statistic):                      0.0136
Time:                        10:16:46   Log-Likelihood:                         -19.173
No. Observations:                   5   AIC:                                      42.35
Df Residuals:                       3   BIC:                                      41.57
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels.iv.model as lm

houseprices = sm.datasets.get_rdataset(dataname="HousePrices", package="AER", cache=True).data
print(houseprices.iloc[:, list(range(3)) + [5] + [10]].head())

mlr1 = smf.ols(formula="price ~ lotsize + bedrooms", data=houseprices).fit()

mdatac = sm.add_constant(data=houseprices, prepend=False)
mlr2 = lm.IV2SLS(dependent=mdatac["price"], exog=mdatac[["const", "bedrooms"]], endog=mdatac["lotsize"], instruments=mdatac[["driveway", "garage"]]).fit(cov_type="homoskedastic", debiased=True)

print(mlr2.wu_hausman())

In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels.iv.model as lm


# Datos de entrada
x = np.array([3, 7, -4, 5, -11])
z = np.array([2, 4, -4, 8, -10])
y = np.array([20, 30, -50, 60, -60])

mlr2 = lm.IV2SLS(dependent=y, exog=z, endog=x, instruments=z).fit(cov_type="homoskedastic", debiased=True)

print(mlr2.wu_hausman())

ValueError: instruments [exog instruments]  do not have full column rank

In [15]:
import numpy as np
from linearmodels.iv import IV2SLS

# Datos de entrada
x = np.array([3, 7, -4, 5, -11])      # Variable endógena
z = np.array([2, 4, -4, 8, -10])      # Instrumento
y = np.array([20, 30, -50, 60, -60])  # Variable dependiente

# Ajuste del modelo IV con 2SLS
mlr2 = IV2SLS(
    dependent=y,      # Variable dependiente
    exog=None,        # Sin variables exógenas
    endog=x,          # Variable endógena
    instruments=z     # Instrumento para la variable endógena
).fit(cov_type="unadjusted", debiased=True)

print(mlr2.wu_hausman())

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 5.1579
P-value: 0.1078
Distributed: F(1,3)


In [17]:
import numpy as np

# Datos
X = np.array([
    [1, 2],
    [1, 2],
    [1, -5],
    [1, 5],
    [1, -4]
])  # Matriz X
y = np.array([4, 6, -9, 11, -7])  # Vector y

# Paso 1: Calcular (X'X)^(-1)
XtX_inv = np.linalg.inv(X.T @ X)

# Paso 2: Calcular estimadores OLS: β̂ = (X'X)^(-1) X'y
beta_hat = XtX_inv @ (X.T @ y)

# Paso 3: Calcular residuos: ê = y - Xβ̂
residuals = y - X @ beta_hat

# Paso 4: Estimar sigma^2 insesgado: σ̂^2 = (ê'ê) / (n - k)
n, k = X.shape  # Número de observaciones y parámetros
sigma2_hat = (residuals.T @ residuals) / (n - k)

# Paso 5: Estimar Var(β̂2): σ̂^2 * [(X'X)^(-1)](2,2)
var_beta2_hat = sigma2_hat * XtX_inv[1, 1]

# Resultados
print("Coeficientes estimados (beta_hat):", beta_hat)
print("Estimador insesgado de sigma^2:", sigma2_hat)
print("Varianza insesgada de beta_2:", var_beta2_hat)



Coeficientes estimados (beta_hat): [1. 2.]
Estimador insesgado de sigma^2: 0.6666666666666666
Varianza insesgada de beta_2: 0.009009009009009009


In [18]:
import numpy as np
import statsmodels.api as sm

# Datos
X = np.array([
    [1, 2],
    [1, 2],
    [1, -5],
    [1, 5],
    [1, -4]
])  # Matriz X
y = np.array([4, 6, -9, 11, -7])  # Vector y

# Ajuste del modelo OLS con statsmodels
model = sm.OLS(y, X).fit()

# Obtener estimador insesgado de sigma^2
sigma2_hat = model.mse_resid

# Extraer la matriz (X'X)^(-1) del modelo
XtX_inv = model.normalized_cov_params

# Varianza de beta_2 (segundo coeficiente)
var_beta2_hat = sigma2_hat * XtX_inv[1, 1]

# Resultados
print("Coeficientes estimados (beta_hat):", model.params)
print("Estimador insesgado de sigma^2:", sigma2_hat)
print("Varianza insesgada de beta_2:", var_beta2_hat)


Coeficientes estimados (beta_hat): [1. 2.]
Estimador insesgado de sigma^2: 0.6666666666666666
Varianza insesgada de beta_2: 0.00900900900900901


In [1]:
import numpy as np
import statsmodels.api as sm

# Datos
X = np.array([
    [1, 2],
    [1, 2],
    [1, -5],
    [1, 5],
    [1, -5]
])  # Matriz X
y = np.array([4, 6, -9, 11, -7])  # Vector y

# Ajuste del modelo OLS con statsmodels
model = sm.OLS(y, X).fit()

# Obtener estimador insesgado de sigma^2
sigma2_hat = model.mse_resid

# Extraer la matriz (X'X)^(-1) del modelo
XtX_inv = model.normalized_cov_params

# Varianza de beta_2 (segundo coeficiente)
var_beta2_hat = sigma2_hat * XtX_inv[1, 1]

# Resultados
print("Coeficientes estimados (beta_hat):", model.params)
print("Estimador insesgado de sigma^2:", sigma2_hat)
print("Varianza insesgada de beta_2:", var_beta2_hat)

Coeficientes estimados (beta_hat): [1.37681159 1.88405797]
Estimador insesgado de sigma^2: 1.3623188405797102
Varianza insesgada de beta_2: 0.01645312609395786


In [1]:
import numpy as np
np.sqrt(0.7)

0.8366600265340756

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Crear datos simulados
np.random.seed(42)
n = 100
X = np.random.rand(n, 1)  # Variable independiente
Y = 3 * X.squeeze() + np.random.normal(0, 0.2, n)  # Variable dependiente con algo de ruido

# Convertir X a un DataFrame para evitar advertencias
X_df = pd.DataFrame(X, columns=["X"])

# Modelo con intercepto
model_with_intercept = LinearRegression()
model_with_intercept.fit(X, Y)
y_pred_with_intercept = model_with_intercept.predict(X)
residuals_with_intercept = Y - y_pred_with_intercept

# Modelo sin intercepto
model_without_intercept = LinearRegression(fit_intercept=False)
model_without_intercept.fit(X, Y)
y_pred_without_intercept = model_without_intercept.predict(X)
residuals_without_intercept = Y - y_pred_without_intercept

# Sumar residuos
sum_residuals_with_intercept = np.sum(residuals_with_intercept)
sum_residuals_without_intercept = np.sum(residuals_without_intercept)

# Imprimir resultados
print("Suma de los residuos (con intercepto):", sum_residuals_with_intercept)
print("Suma de los residuos (sin intercepto):", sum_residuals_without_intercept)

# Analizar diferencias
print("Diferencia en la suma de residuos:", sum_residuals_with_intercept - sum_residuals_without_intercept)


Suma de los residuos (con intercepto): 1.3877787807814457e-14
Suma de los residuos (sin intercepto): 1.2210276724038787
Diferencia en la suma de residuos: -1.221027672403865


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Crear datos simulados
np.random.seed(42)
n = 100
X1 = np.random.rand(n, 1)  # Primer regresor
X2 = np.random.rand(n, 1)  # Segundo regresor
Y = 2 * X1.squeeze() - 3 * X2.squeeze() + np.random.normal(0, 0.2, n)  # Variable dependiente con ruido

# Combinar regresores
X = np.hstack([X1, X2])

# Modelo completo con todos los regresores
model_full = LinearRegression()
model_full.fit(X, Y)
coefficients_full = model_full.coef_
print("Coeficientes del modelo completo:", coefficients_full)

# Regresión de Y contra X2 para obtener residuos (parcialización del regresando)
model_Y_X2 = LinearRegression()
model_Y_X2.fit(X2, Y)
residuals_Y = Y - model_Y_X2.predict(X2)

# Regresión de X1 contra X2 para obtener residuos (parcialización del regresor)
model_X1_X2 = LinearRegression()
model_X1_X2.fit(X2, X1)
residuals_X1 = X1.squeeze() - model_X1_X2.predict(X2)

# Regresión de los residuos de Y contra los residuos de X1
model_residuals = LinearRegression()
model_residuals.fit(residuals_X1, residuals_Y)
coef_residuals = model_residuals.coef_[0]

print("Coeficiente de la regresión parcial (residuos):", coef_residuals)

# Comparar signo con la correlación parcial
correlation_partial = np.corrcoef(residuals_Y, residuals_X1)[0, 1]
print("Correlación parcial entre Y y X1:", correlation_partial)
print("Signos coinciden:", np.sign(coef_residuals) == np.sign(correlation_partial))


Coeficientes del modelo completo: [ 1.93165494 -2.85613772]
Coeficiente de la regresión parcial (residuos): -23499388839769.58
Correlación parcial entre Y y X1: 0.9461216502399327
Signos coinciden: False


In [2]:
import numpy as np
import statsmodels.api as sm

# Datos
x = np.array([1, 2, -1])
y = np.array([0, 2, 1])

# Modelo de regresión
model = sm.OLS(y, x).fit()

# Resultados con errores estándar robustos de White
robust_cov = model.get_robustcov_results(cov_type='HC0')

# Imprimir resumen del modelo con errores estándar robustos
print(robust_cov.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.300
Model:                            OLS   Adj. R-squared (uncentered):             -0.050
Method:                 Least Squares   F-statistic:                              1.385
Date:                Sun, 15 Dec 2024   Prob (F-statistic):                       0.360
Time:                        21:40:33   Log-Likelihood:                         -4.4880
No. Observations:                   3   AIC:                                      10.98
Df Residuals:                       2   BIC:                                      10.07
Df Model:                           1                                                  
Covariance Type:                  HC0                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [5]:
import numpy as np

# Definir los datos
Y = np.array([20, 30, -50, 60, -60])
x = np.array([3, 7, -4, 5, -11])
z = np.array([1, 2, -2, 4, -5])

# Primera regresión: x en función de z
beta_x_z = np.sum(z * x) / np.sum(z * z)  # Coeficiente
xsombrero = z * beta_x_z  # Valores ajustados (xsombrero)

# Segunda regresión: y en función de x y xsombrero
X_combined = np.column_stack((x, xsombrero))  # Combinar x y xsombrero
beta_y_x_xsombrero = np.linalg.inv(X_combined.T @ X_combined) @ X_combined.T @ Y  # Coeficientes

# Resultados
print("Resultados de la primera regresión (x ~ z):")
print(f"Coeficiente: {beta_x_z}")

print("\nResultados de la segunda regresión (y ~ x + xsombrero):")
print(f"Coeficientes: {beta_y_x_xsombrero}")

Resultados de la primera regresión (x ~ z):
Coeficiente: 2.0

Resultados de la segunda regresión (y ~ x + xsombrero):
Coeficientes: [-0.5  7.7]


In [8]:
100 * np.linalg.inv(X_combined.T @ X_combined)

array([[ 5. , -5. ],
       [-5. ,  5.5]])

In [9]:
7.7 / np.sqrt(5.5)

3.283291031876401

In [11]:
import numpy as np

# Definir los datos
Y = np.array([20, 30, -50, 60, -60])
x = np.array([3, 7, -4, 5, -11])
z = np.array([1, 2, -2, 4, -5])

# Primera regresión: x en función de z
beta_x_z = np.sum(z * x) / np.sum(z * z)  # Coeficiente
xsombrero = z * beta_x_z  # Valores ajustados (xsombrero)

# Segunda regresión: y en función de x y xsombrero
X_combined = np.column_stack((x, xsombrero))  # Combinar x y xsombrero
beta_y_x_xsombrero = np.linalg.inv(X_combined.T @ X_combined) @ X_combined.T @ Y  # Coeficientes

# Promedio de Y
promedio_Y = np.mean(Y)

# Regresión con término de ordenada (intercepto)
X_with_intercept = np.column_stack((np.ones(len(x)), x))  # Agregar constante
beta_with_intercept = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ Y

# Regresión sin término de ordenada
beta_no_intercept = np.sum(x * Y) / np.sum(x * x)

# Resultados
print("Resultados de la primera regresión (x ~ z):")
print(f"Coeficiente: {beta_x_z}")

print("\nResultados de la segunda regresión (y ~ x + xsombrero):")
print(f"Coeficientes: {beta_y_x_xsombrero}")

print("\nPromedio de Y:")
print(f"Promedio: {promedio_Y}")

print("\nRegresión con término de ordenada (y ~ 1 + x):")
print(f"Coeficientes: {beta_with_intercept}")

print("\nRegresión sin término de ordenada (y ~ x):")
print(f"Coeficiente: {beta_no_intercept}")

# Discusión
if promedio_Y == 0:
    print("\nEl promedio de Y es cero. Esto ocurre cuando los valores de Y están simétricamente distribuidos alrededor del origen.")
else:
    print("\nEl promedio de Y no es cero. Esto indica que hay un sesgo en los valores de Y respecto al origen.")

Resultados de la primera regresión (x ~ z):
Coeficiente: 2.0

Resultados de la segunda regresión (y ~ x + xsombrero):
Coeficientes: [-0.5  7.7]

Promedio de Y:
Promedio: 0.0

Regresión con término de ordenada (y ~ 1 + x):
Coeficientes: [-6.66133815e-16  6.50000000e+00]

Regresión sin término de ordenada (y ~ x):
Coeficiente: 6.5

El promedio de Y es cero. Esto ocurre cuando los valores de Y están simétricamente distribuidos alrededor del origen.
